# Changes in OPF cost formulation
With the new pandapower Version 1.5.1, the cost formulation for the pandapower OPF changes. Basically, you'll be good if you just multiply all your cost functions with -1 and your results will be the same as before. Maybe you already realized yourself, that there was an inconsistency with the costs. 

This tutorial will explain why we made this change and how you can refactor your old code. It is based on the opf_basic tutorial.
So let's remember one convention in pandapower: 
For all bus-based power values, the signing is based on the consumer viewpoint:

- positive active power is power consumption, negative active power is power generation
- positive reactive power is inductive consumption, negative reactive power is capacitive consumption

Let's say I wan't to define linear costs of 1€ per kW for a generator, then I need to keep this in mind!
The respective cost function has a negative slope:

<img src="pics/cost1.png" width="25%">

In pandapower this could be created with the following commands:
    - pp.create_polynomial_cost(net, 1, 'gen', np.array([-1, 0]))
or
    - pp.create_piecewise_linear_cost(net, 1, "gen", np.array([[p_min_kw, abs(p_min_kw)], [0,0]]))
E.g. when we have a 1000 kW generator:    
    - pp.create_piecewise_linear_cost(net, 1, "gen", np.array([[-1000, 1000], [0,0]]))
 

Let's start with the examples. We first create the grid, where nothing has changed:

In [1]:
import pandapower as pp
import numpy as np
net = pp.create_empty_network()

#create buses
bus1 = pp.create_bus(net, vn_kv=220.)
bus2 = pp.create_bus(net, vn_kv=110.)
bus3 = pp.create_bus(net, vn_kv=110.)
bus4 = pp.create_bus(net, vn_kv=110.)

#create 220/110 kV transformer
pp.create_transformer(net, bus1, bus2, std_type="100 MVA 220/110 kV")

#create 110 kV lines
pp.create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
pp.create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')

#create loads
pp.create_load(net, bus2, p_kw=60e3, controllable = False)
pp.create_load(net, bus3, p_kw=70e3, controllable = False)
pp.create_load(net, bus4, p_kw=10e3, controllable = False)

#create generators
eg = pp.create_ext_grid(net, bus1)
g0 = pp.create_gen(net, bus3, p_kw=-80*1e3, min_p_kw=-80e3, max_p_kw=0,vm_pu=1.01, controllable=True)
g1 = pp.create_gen(net, bus4, p_kw=-100*1e3, min_p_kw=-100e3, max_p_kw=0, vm_pu=1.01, controllable=True)

## Loss Minimization

We specify the same costs for the power at the external grid and all generators to minimize the overall power feed in. This equals an overall loss minimization. With the former pandapower versions, you must've specified the costs like this to get to the goal of having positive costs for high feed in: 
    - costeg = pp.create_polynomial_cost(net, 0, 'ext_grid', np.array([1, 0]))
    - costgen1 = pp.create_polynomial_cost(net, 0, 'gen', np.array([1, 0]))
    - costgen2 = pp.create_polynomial_cost(net, 1, 'gen', np.array([1, 0]))
    
But as we leared above, this is not right! This was due to a mix up in the signs, but from now on, the cost function signing is based on the consumer viewpoint as well.

In [2]:
costeg = pp.create_polynomial_cost(net, 0, 'ext_grid', np.array([-1, 0]))
costgen1 = pp.create_polynomial_cost(net, 0, 'gen', np.array([-1, 0]))
costgen2 = pp.create_polynomial_cost(net, 1, 'gen', np.array([-1, 0]))

We run an OPF:

In [3]:
pp.runopp(net, verbose=True)

hp.pandapower.run - INFO: These missing columns in ext_grid are considered in OPF as +- 1000 TW.: ['min_p_kw' 'max_p_kw' 'min_q_kvar' 'max_q_kvar']
hp.pandapower.run - INFO: These elements have missing power constraint values, which are considered in OPF as +- 1000 TW: ['gen']
hp.pandapower.run - INFO: min_vm_pu is missing in bus table. In OPF these limits are considered as 0.0 pu.
hp.pandapower.run - INFO: max_vm_pu is missing in bus table. In OPF these limits are considered as 2.0 pu.


PYPOWER Version 5.0.0, 29-May-2015 -- AC Optimal Power Flow
Python Interior Point Solver - PIPS, Version 1.0, 07-Feb-2011
Converged!


let's check the results:

In [4]:
net.res_ext_grid

,p_kw,q_kvar
0,-56530.133524,-1974.471614


In [5]:
net.res_gen

,p_kw,q_kvar,va_degree,vm_pu
0,-71313.545821,1969.654618,-3.712804,1.000009
1,-12299.610412,1451.160058,-3.712782,1.000010


Since all costs were specified the same, the OPF minimizes overall power generation, which is equal to a loss minimization in the network. The loads at buses 3 and 4 are supplied by generators at the same bus, the load at Bus 2 is provided by a combination of the other generators so that the power transmission leads to minimal losses.

What about the cost result? We expect costs of 1 for each kW fed into the net. So let's check the overall fed in power:

In [12]:
net.res_gen.p_kw.sum() + net.res_ext_grid.p_kw.sum()

-140143.2897573685

So the cost value of the OPF must be 140143.29!

In [14]:
net.res_cost

140143.2897573685

We have some consistency in our OPF cost functions now!